In [4]:
import numpy as np
import pandas as pd
# from sklearn.ensemble import RandomForestClassifier

In [ ]:
data = pd.read_csv("Life Expectancy Data Cleaned With Insurance.csv")
countries = np.unique(data['Country'])
regions = np.unique(data["Region"])
def get_idx(country):
    return np.where(countries == country)[0][0]
def get_region_idx(region):
    return np.where(regions == region)[0][0]

# Replace all country names with indexes

# Replace all region names with region indexes

# Take out the "Life Expectancy" column as y
# The rest of the data is X

# Create Random forest
# forest = RandomForestClassifier()
# forest.fit(X,y)


139
      Unnamed: 0 Country          Region  Year  Infant_deaths  \
2423        2423  Sweden  European Union  2004            3.1   

      Under_five_deaths  Adult_mortality  Alcohol_consumption  Hepatitis_B  \
2423                3.7          66.1935                 6.51           60   

      Measles  ...  Incidents_HIV  GDP_per_capita  Population_mln  \
2423       95  ...           0.08           44891            8.99   

      Thinness_ten_nineteen_years  Thinness_five_nine_years  Schooling  \
2423                          1.3                       1.3       12.3   

      Economy_status_Developed  Economy_status_Developing  Life_expectancy  \
2423                         1                          0             80.5   

      Universal_Insurance  
2423                 True  

[1 rows x 23 columns]
